In [25]:
import pandas as pd
import pickle
import numpy as np

In [90]:
df = pickle.load(open('dataset_level2.pkl','rb'))

In [91]:
df

# batting_team
# bowling team
# city
# current score
# ball left
# wickets left
# current rr
# last five

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,2,Australia,Sri Lanka,0.1,0,0,NaN,Melbourne Cricket Ground
1,2,Australia,Sri Lanka,0.2,0,0,NaN,Melbourne Cricket Ground
2,2,Australia,Sri Lanka,0.3,1,0,NaN,Melbourne Cricket Ground
3,2,Australia,Sri Lanka,0.4,2,0,NaN,Melbourne Cricket Ground
4,2,Australia,Sri Lanka,0.5,0,0,NaN,Melbourne Cricket Ground
...,...,...,...,...,...,...,...,...
121,964,Sri Lanka,Australia,19.3,1,0,Colombo,R Premadasa Stadium
122,964,Sri Lanka,Australia,19.4,0,0,Colombo,R Premadasa Stadium
123,964,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,R Premadasa Stadium
124,964,Sri Lanka,Australia,19.6,2,0,Colombo,R Premadasa Stadium


In [92]:
df.isnull().sum()

match_id                0
batting_team            0
bowling_team            0
ball                    0
runs                    0
player_dismissed        0
city                15701
venue                   0
dtype: int64

In [93]:
df[df['city'].isnull()]['venue'].value_counts()

Dubai International Cricket Stadium        5034
Harare Sports Club                         2731
Pallekele International Cricket Stadium    2066
Sharjah Cricket Stadium                    1499
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       749
Sylhet Stadium                              740
Adelaide Oval                               498
Mombasa Sports Club Ground                  371
Rawalpindi Cricket Stadium                  368
Sylhet International Cricket Stadium        128
Carrara Oval                                 64
Name: venue, dtype: int64

In [94]:
cities = np.where(df['city'].isnull(),df['venue'].str.split().apply(lambda x:x[0]),df['city'])


In [95]:
df['city'] = cities
df.isnull().sum()

match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [86]:
df.drop(columns=['venue'],inplace=True)
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne


In [77]:
eligible_cities = df['city'].value_counts()[df['city'].value_counts() > 600].index.tolist()
df = df[df['city'].isin(eligible_cities)]
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr,last_five
0,2,Australia,Sri Lanka,0.1,0,1,Melbourne,0,0,1,1,119,9,0.0,NaN
1,2,Australia,Sri Lanka,0.2,0,2,Melbourne,0,0,2,2,118,8,0.0,NaN
2,2,Australia,Sri Lanka,0.3,1,3,Melbourne,1,0,3,3,117,7,2.0,NaN
3,2,Australia,Sri Lanka,0.4,2,4,Melbourne,3,0,4,4,116,6,4.5,NaN
4,2,Australia,Sri Lanka,0.5,0,5,Melbourne,3,0,5,5,115,5,3.6,NaN


In [73]:
df['current_score'] = df.groupby('match_id').cumsum()['runs']
df['over'] = df['ball'].apply(lambda x:str(x).split(".")[0])
df['ball_no'] = df['ball'].apply(lambda x:str(x).split(".")[1])
df['balls_bowled'] = (df['over'].astype('int')*6) + df['ball_no'].astype('int')
df.head()

C:\Users\deban\AppData\Local\Temp\ipykernel_24136\1131127802.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.cumsum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df['current_score'] = df.groupby('match_id').cumsum()['runs']
C:\Users\deban\AppData\Local\Temp\ipykernel_24136\1131127802.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['current_score'] = df.groupby('match_id').cumsum()['runs']
C:\Users\deban\AppData\Local\Temp\ipykernel_24136\1131127802.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr,last_five
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,1,119,10,0.0,NaN
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,2,118,10,0.0,NaN
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3,117,10,2.0,NaN
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,4,116,10,4.5,NaN
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,5,115,10,3.6,NaN


In [69]:
df['balls_left'] = 120 - df['balls_bowled']
df['balls_left'] = df['balls_left'].apply(lambda x:0 if x<0 else x)
df.head()

C:\Users\deban\AppData\Local\Temp\ipykernel_24136\3614643678.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['balls_left'] = 120 - df['balls_bowled']
C:\Users\deban\AppData\Local\Temp\ipykernel_24136\3614643678.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['balls_left'] = df['balls_left'].apply(lambda x:0 if x<0 else x)


,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr,last_five
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,1,119,10,0.0,NaN
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,2,118,10,0.0,NaN
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3,117,10,2.0,NaN
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,4,116,10,4.5,NaN
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,5,115,10,3.6,NaN


In [74]:
df['player_dismissed'] = df['player_dismissed'].apply(lambda x:0 if x=='0' else 1)
df['player_dismissed'] = df['player_dismissed'].astype('int')
df['player_dismissed'] = df.groupby('match_id').cumsum()['player_dismissed']
df['wickets_left'] = 10 - df['player_dismissed']
df.head()

C:\Users\deban\AppData\Local\Temp\ipykernel_24136\377913442.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['player_dismissed'] = df['player_dismissed'].apply(lambda x:0 if x=='0' else 1)
C:\Users\deban\AppData\Local\Temp\ipykernel_24136\377913442.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['player_dismissed'] = df['player_dismissed'].astype('int')
C:\Users\deban\AppData\Local\Temp\ipykernel_24136\377913442.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.cumsum i

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr,last_five
0,2,Australia,Sri Lanka,0.1,0,1,Melbourne,0,0,1,1,119,9,0.0,NaN
1,2,Australia,Sri Lanka,0.2,0,2,Melbourne,0,0,2,2,118,8,0.0,NaN
2,2,Australia,Sri Lanka,0.3,1,3,Melbourne,1,0,3,3,117,7,2.0,NaN
3,2,Australia,Sri Lanka,0.4,2,4,Melbourne,3,0,4,4,116,6,4.5,NaN
4,2,Australia,Sri Lanka,0.5,0,5,Melbourne,3,0,5,5,115,5,3.6,NaN


In [75]:
df['crr'] = (df['current_score']*6)/df['balls_bowled']
df.head()

C:\Users\deban\AppData\Local\Temp\ipykernel_24136\3408492214.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['crr'] = (df['current_score']*6)/df['balls_bowled']


,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr,last_five
0,2,Australia,Sri Lanka,0.1,0,1,Melbourne,0,0,1,1,119,9,0.0,NaN
1,2,Australia,Sri Lanka,0.2,0,2,Melbourne,0,0,2,2,118,8,0.0,NaN
2,2,Australia,Sri Lanka,0.3,1,3,Melbourne,1,0,3,3,117,7,2.0,NaN
3,2,Australia,Sri Lanka,0.4,2,4,Melbourne,3,0,4,4,116,6,4.5,NaN
4,2,Australia,Sri Lanka,0.5,0,5,Melbourne,3,0,5,5,115,5,3.6,NaN


In [47]:
groups = df.groupby('match_id')

match_ids = df['match_id'].unique()
last_five = []
for id in match_ids:
    last_five.extend(groups.get_group(id).rolling(window=30).sum()['runs'].values.tolist())

C:\Users\deban\AppData\Local\Temp\ipykernel_24136\3055083752.py:6: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['batting_team', 'bowling_team', 'city'], dtype='object')
  last_five.extend(groups.get_group(id).rolling(window=30).sum()['runs'].values.tolist())


In [48]:
df['last_five'] = last_five

C:\Users\deban\AppData\Local\Temp\ipykernel_24136\1595651733.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_five'] = last_five


In [76]:
df[['batting_team','bowling_team','city','current_score','balls_left','wickets_left','crr','last_five']]
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr,last_five
0,2,Australia,Sri Lanka,0.1,0,1,Melbourne,0,0,1,1,119,9,0.0,NaN
1,2,Australia,Sri Lanka,0.2,0,2,Melbourne,0,0,2,2,118,8,0.0,NaN
2,2,Australia,Sri Lanka,0.3,1,3,Melbourne,1,0,3,3,117,7,2.0,NaN
3,2,Australia,Sri Lanka,0.4,2,4,Melbourne,3,0,4,4,116,6,4.5,NaN
4,2,Australia,Sri Lanka,0.5,0,5,Melbourne,3,0,5,5,115,5,3.6,NaN


In [50]:
final_df = df.groupby('match_id').sum()['runs'].reset_index().merge(df,on='match_id')

C:\Users\deban\AppData\Local\Temp\ipykernel_24136\3687880561.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  final_df = df.groupby('match_id').sum()['runs'].reset_index().merge(df,on='match_id')


In [51]:
final_df=final_df[['batting_team','bowling_team','city','current_score','balls_left','wickets_left','crr','last_five','runs_x']]

In [52]:
final_df.dropna(inplace=True)

In [53]:
final_df.isnull().sum()

batting_team     0
bowling_team     0
city             0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_five        0
runs_x           0
dtype: int64

In [54]:
final_df = final_df.sample(final_df.shape[0])

In [55]:
final_df.sample(2)

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
32878,South Africa,New Zealand,London,53,66,9,5.888889,25.0,128
21997,Scotland,Netherlands,Dubai,127,5,3,6.626087,35.0,130


In [56]:
X = final_df.drop(columns=['runs_x'])
y = final_df['runs_x']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [57]:
X_train

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five
68510,England,Pakistan,Manchester,108,27,5,6.967742,27.0
53038,Pakistan,Australia,Dubai,56,65,9,6.109091,29.0
54418,South Africa,Sri Lanka,Hambantota,135,5,4,7.043478,42.0
41210,Afghanistan,Ireland,Dubai,33,83,8,5.351351,27.0
29399,Sri Lanka,Australia,Cape Town,82,24,3,5.125000,26.0
...,...,...,...,...,...,...,...,...
18395,Nepal,Malaysia,Kuala Lumpur,118,26,5,7.531915,43.0
63590,New Zealand,Pakistan,Dubai,127,10,4,6.927273,45.0
60536,Australia,West Indies,Mirpur,112,39,5,8.296296,39.0
55298,South Africa,Australia,Centurion,82,32,4,5.590909,24.0


In [58]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error

In [59]:
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','city'])
]
,remainder='passthrough')

In [60]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=12,random_state=1))
])

In [96]:
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(r2_score(y_test,y_pred)*100,"%")

98.8908631241899 %


In [ ]:
pickle.dump(pipe,open('pipe.pkl','wb'))

In [62]:
df["city"].unique()

array(['Melbourne', 'Adelaide', 'Harare', 'Mount Maunganui', 'Auckland',
       'Southampton', 'Cardiff', 'Nagpur', 'Bangalore', 'Greater Noida',
       'Lauderhill', 'Dubai', 'Abu Dhabi', 'Sydney', 'Wellington',
       'Hamilton', 'Barbados', 'Trinidad', 'Colombo', 'St Kitts',
       'Manchester', 'Delhi', 'Lahore', 'Johannesburg', 'Centurion',
       'Cape Town', 'Mumbai', 'Edinburgh', 'Dhaka', 'Sylhet', 'Sharjah',
       'Dublin', 'London', 'Rotterdam', 'Kolkata', 'Al Amarat', 'Durban',
       'Chandigarh', 'Christchurch', 'Kuala Lumpur', 'Nottingham',
       'Guyana', 'St Lucia', 'Pallekele', 'Mirpur', 'Hambantota',
       'Chittagong'], dtype=object)

In [63]:
df["batting_team"].unique()

array(['Australia', 'Zimbabwe', 'India', 'New Zealand', 'South Africa',
       'England', 'Ireland', 'Afghanistan', 'West Indies', 'Pakistan',
       'United Arab Emirates', 'Scotland', 'Oman', 'Hong Kong',
       'Bangladesh', 'Sri Lanka', 'Netherlands', 'Nepal', 'Malaysia',
       'Kenya'], dtype=object)

In [64]:
X_test.isnull().sum()

batting_team     0
bowling_team     0
city             0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_five        0
dtype: int64

In [65]:
x1 = {
    "batting_team": "India",
    "bowling_team": "Pakistan",
    "city": "Melbourne",
    "current_score": 100,
    "balls_left": 47,
    "wickets_left": 5,
    "crr": 9.54,
    "last_five": 53
}

# Convert the dictionary to DataFrame
d = pd.DataFrame([x1])

# Display the DataFrame
print(d)

  batting_team bowling_team       city  current_score  balls_left  \
0        India     Pakistan  Melbourne            100          47   

   wickets_left   crr  last_five  
0             5  9.54         53  


In [66]:
y_pr= pipe.predict(d)
print(y_pr)


[167.55983]
